# Анализ текста на SVM

In [1]:
import pandas as pd
import numpy as np
import sklearn.model_selection
import sklearn.datasets
import sklearn.svm
import sklearn.feature_extraction.text
import sklearn.model_selection

Загрузим объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм"

In [2]:
newsgroups = sklearn.datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )
x = newsgroups.data
y = newsgroups.target

Вычислим TF-IDF-признаки для всех текстов

In [3]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer()
x_vect = vectorizer.fit_transform(x)

Подберем минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром при помощи кросс-валидации по 5 блокам. В качестве меры качества используем долю верных ответов (accuracy)

In [4]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}  # c parameter grid
kf = sklearn.model_selection.KFold(n_splits=5, shuffle=True, random_state=0)  # cross validation
clf = sklearn.svm.SVC(kernel='linear', random_state=0)  # SVM classifier
gs = sklearn.model_selection.GridSearchCV(clf, grid, scoring='accuracy', cv=kf)  # grid search
gs.fit(x_vect, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(kernel='linear', random_state=0),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [5]:
coptdict = gs.best_params_  # optimal с values dict
copt = list(coptdict.values())[0]  # optimal с value
print(f'Optimal value С = {copt}')

Optimal value С = 10.0


Обучим SVM по всей выборке с оптимальным параметром C

In [6]:
clf = sklearn.svm.SVC(C=copt, kernel='linear', random_state=0)
clf.fit(x_vect, y)

SVC(C=10.0, kernel='linear', random_state=0)

Найдем 10 слов с наибольшим абсолютным значением веса

In [7]:
weights = abs(clf.coef_.todense())  # from sparse matrix and abs val
indexes = np.asarray(np.argsort(weights))[0]
top10inds = indexes[-10:]
top10words = []
for ind in top10inds:
    word = vectorizer.get_feature_names()[ind]
    top10words.append(word)
print(top10words)

['nick', 'keith', 'bible', 'religion', 'sky', 'moon', 'atheists', 'atheism', 'god', 'space']
